<font size="5">**Project Summary**</font><br>
In this competition, the main task is to do surface time series classification. 1d convolution is a good technique when doing the time series classification job. So I apply 1d convolutional neural network to solve this problem. The whole code is written in Pytorch.

<font size="5">**External libs we are using**</font><br>

In [53]:
# libs we are using
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

<font size="5">**Data preparations**</font><br>

In this project, I use the raw data as input of the network. I concatenated all datasets into one single numpy array. The first 3810 rows are training data with given labels, the rest 3816 rows are testing data where labels are not given. In total raw data has 7816 rows. In addition to that, we split the raw training data into training data and validation data by the ratio of 80/20.


I create three datasets and data loaders for them to make the data ready for model's training. The process is straightforward. I split the labelled data into two subsets, and keep testing data as is. Also, I convert Numpy arrays into `torch.tensor` objects of proper type (float for samples, and long - for targets).


In [54]:
def create_datasets(data, target, train_size, valid_pct=0.2, seed=None):
    """Converts NumPy arrays into PyTorch datsets.
    
    Three datasets are created in total:
        * training dataset
        * validation dataset
        * testing (un-labelled) dataset

    """
    raw =  data
    sz = train_size
    idx = np.arange(sz)
    trn_idx, val_idx = train_test_split(
        idx, test_size=valid_pct, random_state=seed)
     
    trn_ds = TensorDataset(
        torch.tensor(raw[:sz][trn_idx]).float(), 
        torch.tensor(target[:sz][trn_idx]).long())
    val_ds = TensorDataset(
        torch.tensor(raw[:sz][val_idx]).float(), 
        torch.tensor(target[:sz][val_idx]).long())
    tst_ds = TensorDataset(
        torch.tensor(raw[sz:]).float(), 
        torch.tensor(target[sz:]).long())
    return trn_ds, val_ds, tst_ds

# We use this function to divide the data
def create_datasets2(data_arr, train_size, valid_pct=0.2, seed=None):
    """Converts NumPy arrays into PyTorch datsets.
    
    Three datasets are created in total:
        * training dataset
        * validation dataset
        * testing (un-labelled) dataset

    """
    raw =  data_arr
    sz = train_size

    idx = np.arange(sz)
    print("total data length")
    print(len(idx))
    trn_idx, tst_idx = train_test_split(idx, test_size=0.2, random_state=1)
    trn_idx, val_idx = train_test_split(trn_idx, test_size=0.25, random_state=1)
    
    print("input one dimension shape")
    print(raw[0].shape)
    print("training data length")
    print(len(trn_idx))
    print("validation data le")
    print(len(val_idx))
    print("testing d")
    print(len(tst_idx))
    
    trn_ds = TensorDataset(
        torch.tensor(raw[0][:sz][trn_idx]).float(),
        torch.tensor(raw[1][:sz][trn_idx]).float(),
        torch.tensor(raw[2][:sz][trn_idx]).float(),
        torch.tensor(raw[3][:sz][trn_idx]).float(),
        torch.tensor(raw[4][:sz][trn_idx]).float(),
        torch.tensor(raw[5][:sz][trn_idx]).float(),
        torch.tensor(raw[6][:sz][trn_idx]).float(),
        torch.tensor(raw[7][:sz][trn_idx]).float(),
        torch.tensor(raw[8][:sz][trn_idx]).float(),
        torch.tensor(target[:sz][trn_idx]).long())
    
    val_ds = TensorDataset(
        torch.tensor(raw[0][:sz][val_idx]).float(),
        torch.tensor(raw[1][:sz][val_idx]).float(), 
        torch.tensor(raw[2][:sz][val_idx]).float(), 
        torch.tensor(raw[3][:sz][val_idx]).float(), 
        torch.tensor(raw[4][:sz][val_idx]).float(), 
        torch.tensor(raw[5][:sz][val_idx]).float(), 
        torch.tensor(raw[6][:sz][val_idx]).float(), 
        torch.tensor(raw[7][:sz][val_idx]).float(), 
        torch.tensor(raw[8][:sz][val_idx]).float(), 
        torch.tensor(target[:sz][val_idx]).long())
    
    tst_ds = TensorDataset(
        torch.tensor(raw[0][:sz][tst_idx]).float(),
        torch.tensor(raw[1][:sz][tst_idx]).float(),
        torch.tensor(raw[2][:sz][tst_idx]).float(),
        torch.tensor(raw[3][:sz][tst_idx]).float(),
        torch.tensor(raw[4][:sz][tst_idx]).float(),
        torch.tensor(raw[5][:sz][tst_idx]).float(),
        torch.tensor(raw[6][:sz][tst_idx]).float(),
        torch.tensor(raw[7][:sz][tst_idx]).float(),
        torch.tensor(raw[8][:sz][tst_idx]).float(),
        torch.tensor(target[:sz][tst_idx]).long())
    
    return trn_ds, val_ds, tst_ds

def create_loaders(data, bs=128, jobs=0):
    """Wraps the datasets returned by create_datasets function with data loaders."""
    
    trn_ds, val_ds, tst_ds = data
    trn_dl = DataLoader(trn_ds, batch_size=bs, shuffle=True, num_workers=jobs)
    val_dl = DataLoader(val_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    tst_dl = DataLoader(tst_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    return trn_dl, val_dl, tst_dl

<font size="5">**Neural Network Architecture**</font><br>

In [55]:
class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)

In [56]:
class Surface_Classifier(nn.Module):
    def __init__(self, raw_ni, no, drop=.5):
        super().__init__()
        
        self.conv1d_channel_0 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            
           
        
        )
            
            
        
        self.conv1d_channel_1 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
        )
        
        self.conv1d_channel_2 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
        )
        
        self.conv1d_channel_3 = nn.Sequential(
             nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
        )
        
        self.conv1d_channel_4 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
        )
        
        self.conv1d_channel_5 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
        )
        
        self.conv1d_channel_6 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
           
        )
        
        self.conv1d_channel_7 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
        )
            

       
        
        self.conv1d_channel_8 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
        )

            

        self.dense = nn.Sequential(
            nn.Linear(216, 84),  nn.ReLU(),nn.Dropout(),
            nn.Linear(84, 36),  nn.ReLU(),nn.Dropout(),
            nn.Linear(36, no),  nn.ReLU()
        )
        
            

    def forward(self, t_channel_0, t_channel_1, t_channel_2, t_channel_3, t_channel_4, t_channel_5, t_channel_6, t_channel_7, t_channel_8):
        conv1d_out_channel_0 = self.conv1d_channel_0(t_channel_0)
        conv1d_out_channel_1 = self.conv1d_channel_1(t_channel_1)
        conv1d_out_channel_2 = self.conv1d_channel_2(t_channel_2)
        conv1d_out_channel_3 = self.conv1d_channel_3(t_channel_3)
        conv1d_out_channel_4 = self.conv1d_channel_4(t_channel_4)
        conv1d_out_channel_5 = self.conv1d_channel_5(t_channel_5)
        conv1d_out_channel_6 = self.conv1d_channel_6(t_channel_6)
        conv1d_out_channel_7 = self.conv1d_channel_7(t_channel_7)
        conv1d_out_channel_8 = self.conv1d_channel_8(t_channel_8)
        
        t_in = torch.cat([conv1d_out_channel_0,conv1d_out_channel_1, conv1d_out_channel_2, conv1d_out_channel_3, conv1d_out_channel_4, conv1d_out_channel_5, conv1d_out_channel_6, conv1d_out_channel_7, conv1d_out_channel_8], dim=1)
        res = t_in.view(t_in.size(0), -1)
        out = self.dense(res)
        return out
        

<font size="5">**Initial setup for random seed and project path**</font><br>

In [57]:

# set up the seed
seed = 1
np.random.seed(seed)

# path to sample submission
sample = Path.cwd().parent/'input'/'career-con-2019'/'sample_submission.csv'

ROOT = Path.cwd().parent/'input'/'career-con-2019-preprocessed-data'
raw_arr = np.load(ROOT/'feat.npy').transpose(0, 2, 1)
target = np.load(ROOT/'target.npy')

raw_dim_data = [None]*9
#print(raw_dim_data)
for i in range(0, 9):
    raw_dim_data[i] = raw_arr[:,i,:]
#    print("raw data shape")
    
    raw_dim_data[i] = raw_dim_data[i].reshape([7626,1,128])
#    print(raw_dim_data[i].shape)
    
# print("raw array shape")
# print(raw_arr.shape)
# print("label array shape")
# print(target.shape)

trn_sz = 3810  # only the first `trn_sz` rows in each array include labelled data
#datasets = create_datasets((raw_arr), target, trn_sz, seed=seed)
datasets = create_datasets2((raw_dim_data), trn_sz, seed=seed)


# make sure that we run on a proper device (not relevant for Kaggle kernels but helpful in Jupyter sessions)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


trn_dl, val_dl, tst_dl = create_loaders(datasets, bs=256)

total data length
3810
input one dimension shape
(7626, 1, 128)
training data length
2286
validation data le
762
testing d
762


<font size="5">**Training model**</font><br>
Now everything is ready to create a training loop and see if our model works. For each training epoch, the loop performs the following actions:
1. train model on the `trn_ds` dataset;
2. verify quality on the `val_ds` dataset;
3. check if the quality improved since previous epoch, and if so, save the model's weights onto disk;
4. in case if the model's quality isn't impoving for `patience` epochs, the training is stopped.
Also, the code tracks loss and accuracy history, and prints current scores with exponentially increasing logging frequency, i.e., only at 1, 2, 4, 8... epochs.

In [ ]:
raw_feat = raw_arr.shape[1]


lr = 0.002
n_epochs = 10000
iterations_per_epoch = len(trn_dl)
num_classes = 9
best_acc = 0
patience, trials = 500, 0
base = 1
step = 2
loss_history = []
acc_history = []

model = Surface_Classifier(raw_feat, num_classes).to(device)
model.cuda()
criterion = nn.CrossEntropyLoss(reduction='sum')
opt = optim.Adam(model.parameters(), lr=lr)

print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(trn_dl):
        #x_raw, y_batch = [t.to(device) for t in batch]
        x_channel_0, x_channel_1,  x_channel_2,  x_channel_3,  x_channel_4,  x_channel_5,  x_channel_6,  x_channel_7,  x_channel_8, y_batch = [t.to(device) for t in batch]
        opt.zero_grad()
        
#         print("channel 0 type")
#         print(type(x_channel_0))
#         print("channel 0 shape")
#         print(x_channel_0.shape)
#         print("batch type")
#         print(type(batch))
#         print(len(batch))
#         print(batch[0].shape)
#         print(batch[9].shape)

        out = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
        
        
#         out = []
#         with torch.no_grad():
#             for x_channel_0, x_channel_1,  x_channel_2,  x_channel_3,  x_channel_4,  x_channel_5,  x_channel_6,  x_channel_7,  x_channel_8, y_batch in batch:
#                 output = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
#                 out.append(output.detach())
#         out = torch.cat(out)
        

    
        loss = criterion(out, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        opt.step()
        
    epoch_loss /= trn_sz
    loss_history.append(epoch_loss)
    
    model.eval()
    correct, total = 0, 0
    
    for batch in val_dl:
        x_channel_0, x_channel_1,  x_channel_2,  x_channel_3,  x_channel_4,  x_channel_5,  x_channel_6,  x_channel_7,  x_channel_8, y_batch = [t.to(device) for t in batch]
        
        out = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
    acc = correct / total
    acc_history.append(acc)

    if epoch % base == 0:
        print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')
        base *= step
    print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')

    
    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break
            
print('Done!')

Start model training
Epoch:   1. Loss: 1.3133. Acc.: 15.22%
Epoch:   1. Loss: 1.3133. Acc.: 15.22%
Epoch 1 best model saved with accuracy: 15.22%
Epoch:   2. Loss: 1.2846. Acc.: 30.05%
Epoch:   2. Loss: 1.2846. Acc.: 30.05%
Epoch 2 best model saved with accuracy: 30.05%
Epoch:   3. Loss: 1.2596. Acc.: 29.00%
Epoch:   4. Loss: 1.2368. Acc.: 27.17%
Epoch:   4. Loss: 1.2368. Acc.: 27.17%
Epoch:   5. Loss: 1.2173. Acc.: 28.22%
Epoch:   6. Loss: 1.2050. Acc.: 33.99%
Epoch 6 best model saved with accuracy: 33.99%
Epoch:   7. Loss: 1.1831. Acc.: 35.30%
Epoch 7 best model saved with accuracy: 35.30%
Epoch:   8. Loss: 1.1767. Acc.: 36.22%
Epoch:   8. Loss: 1.1767. Acc.: 36.22%
Epoch 8 best model saved with accuracy: 36.22%
Epoch:   9. Loss: 1.1548. Acc.: 35.43%
Epoch:  10. Loss: 1.1363. Acc.: 37.53%
Epoch 10 best model saved with accuracy: 37.53%
Epoch:  11. Loss: 1.1213. Acc.: 36.48%
Epoch:  12. Loss: 1.1060. Acc.: 35.96%
Epoch:  13. Loss: 1.0919. Acc.: 37.93%
Epoch 13 best model saved with ac

Epoch: 149. Loss: 0.6620. Acc.: 66.67%
Epoch: 150. Loss: 0.6459. Acc.: 66.40%
Epoch: 151. Loss: 0.6477. Acc.: 65.88%
Epoch: 152. Loss: 0.6545. Acc.: 67.32%
Epoch: 153. Loss: 0.6451. Acc.: 65.62%
Epoch: 154. Loss: 0.6469. Acc.: 66.14%
Epoch: 155. Loss: 0.6471. Acc.: 66.93%
Epoch: 156. Loss: 0.6397. Acc.: 67.85%
Epoch: 157. Loss: 0.6379. Acc.: 67.32%
Epoch: 158. Loss: 0.6316. Acc.: 66.27%
Epoch: 159. Loss: 0.6251. Acc.: 67.19%
Epoch: 160. Loss: 0.6376. Acc.: 67.59%
Epoch: 161. Loss: 0.6166. Acc.: 68.24%
Epoch 161 best model saved with accuracy: 68.24%
Epoch: 162. Loss: 0.6269. Acc.: 66.80%
Epoch: 163. Loss: 0.6100. Acc.: 66.80%
Epoch: 164. Loss: 0.6143. Acc.: 68.11%
Epoch: 165. Loss: 0.6191. Acc.: 66.40%
Epoch: 166. Loss: 0.6173. Acc.: 65.09%
Epoch: 167. Loss: 0.6094. Acc.: 66.80%
Epoch: 168. Loss: 0.6376. Acc.: 65.88%
Epoch: 169. Loss: 0.6033. Acc.: 66.54%
Epoch: 170. Loss: 0.6097. Acc.: 68.50%
Epoch 170 best model saved with accuracy: 68.50%
Epoch: 171. Loss: 0.6077. Acc.: 65.75%
Epoch

Epoch: 347. Loss: 0.5182. Acc.: 69.69%
Epoch: 348. Loss: 0.5033. Acc.: 70.60%
Epoch: 349. Loss: 0.5142. Acc.: 71.26%
Epoch: 350. Loss: 0.5258. Acc.: 72.18%
Epoch: 351. Loss: 0.5243. Acc.: 73.23%
Epoch: 352. Loss: 0.5164. Acc.: 71.78%
Epoch: 353. Loss: 0.5109. Acc.: 71.65%
Epoch: 354. Loss: 0.5146. Acc.: 71.39%
Epoch: 355. Loss: 0.5109. Acc.: 71.92%
Epoch: 356. Loss: 0.5051. Acc.: 72.18%
Epoch: 357. Loss: 0.5199. Acc.: 72.31%
Epoch: 358. Loss: 0.5193. Acc.: 70.47%
Epoch: 359. Loss: 0.5057. Acc.: 73.36%
Epoch: 360. Loss: 0.5021. Acc.: 72.05%
Epoch: 361. Loss: 0.5082. Acc.: 72.05%
Epoch: 362. Loss: 0.5219. Acc.: 71.65%
Epoch: 363. Loss: 0.5116. Acc.: 72.70%
Epoch: 364. Loss: 0.5113. Acc.: 72.31%
Epoch: 365. Loss: 0.5181. Acc.: 72.83%
Epoch: 366. Loss: 0.5084. Acc.: 73.62%
Epoch: 367. Loss: 0.5167. Acc.: 70.87%
Epoch: 368. Loss: 0.5221. Acc.: 74.80%
Epoch 368 best model saved with accuracy: 74.80%
Epoch: 369. Loss: 0.4995. Acc.: 72.05%
Epoch: 370. Loss: 0.5243. Acc.: 72.05%
Epoch: 371. Los

Epoch: 552. Loss: 0.4723. Acc.: 73.62%
Epoch: 553. Loss: 0.4748. Acc.: 72.97%
Epoch: 554. Loss: 0.4563. Acc.: 72.57%
Epoch: 555. Loss: 0.4544. Acc.: 73.36%
Epoch: 556. Loss: 0.4733. Acc.: 73.75%
Epoch: 557. Loss: 0.4826. Acc.: 73.23%
Epoch: 558. Loss: 0.4565. Acc.: 72.31%
Epoch: 559. Loss: 0.4668. Acc.: 73.36%
Epoch: 560. Loss: 0.4641. Acc.: 71.78%
Epoch: 561. Loss: 0.4825. Acc.: 72.83%
Epoch: 562. Loss: 0.4538. Acc.: 74.28%
Epoch: 563. Loss: 0.4489. Acc.: 76.25%
Epoch: 564. Loss: 0.4609. Acc.: 74.28%
Epoch: 565. Loss: 0.4577. Acc.: 74.28%
Epoch: 566. Loss: 0.4487. Acc.: 75.98%
Epoch: 567. Loss: 0.4552. Acc.: 73.75%
Epoch: 568. Loss: 0.4638. Acc.: 73.23%
Epoch: 569. Loss: 0.4473. Acc.: 73.75%
Epoch: 570. Loss: 0.4612. Acc.: 73.75%
Epoch: 571. Loss: 0.4753. Acc.: 76.12%
Epoch: 572. Loss: 0.4656. Acc.: 72.97%
Epoch: 573. Loss: 0.4391. Acc.: 73.36%
Epoch: 574. Loss: 0.4891. Acc.: 75.20%
Epoch: 575. Loss: 0.4449. Acc.: 73.49%
Epoch: 576. Loss: 0.4858. Acc.: 75.20%
Epoch: 577. Loss: 0.4775.

Epoch: 760. Loss: 0.4310. Acc.: 73.62%
Epoch: 761. Loss: 0.4357. Acc.: 74.80%
Epoch: 762. Loss: 0.4249. Acc.: 76.25%
Epoch: 763. Loss: 0.4380. Acc.: 73.75%
Epoch: 764. Loss: 0.4388. Acc.: 74.02%
Epoch: 765. Loss: 0.4346. Acc.: 75.20%
Epoch: 766. Loss: 0.4465. Acc.: 74.41%
Epoch: 767. Loss: 0.4343. Acc.: 73.88%
Epoch: 768. Loss: 0.4410. Acc.: 74.02%
Epoch: 769. Loss: 0.4477. Acc.: 75.20%
Epoch: 770. Loss: 0.4392. Acc.: 73.23%
Epoch: 771. Loss: 0.4357. Acc.: 74.28%
Epoch: 772. Loss: 0.4446. Acc.: 74.54%
Epoch: 773. Loss: 0.4167. Acc.: 74.02%
Epoch: 774. Loss: 0.4462. Acc.: 74.28%
Epoch: 775. Loss: 0.4395. Acc.: 75.07%
Epoch: 776. Loss: 0.4314. Acc.: 75.72%
Epoch: 777. Loss: 0.4371. Acc.: 74.93%
Epoch: 778. Loss: 0.4305. Acc.: 76.12%
Epoch: 779. Loss: 0.4341. Acc.: 75.59%
Epoch: 780. Loss: 0.4344. Acc.: 74.80%
Epoch: 781. Loss: 0.4314. Acc.: 75.20%
Epoch: 782. Loss: 0.4398. Acc.: 75.85%
Epoch: 783. Loss: 0.4339. Acc.: 74.67%
Epoch: 784. Loss: 0.4392. Acc.: 74.28%
Epoch: 785. Loss: 0.4571.

Epoch: 971. Loss: 0.4082. Acc.: 77.03%
Epoch: 972. Loss: 0.4324. Acc.: 77.82%
Epoch 972 best model saved with accuracy: 77.82%
Epoch: 973. Loss: 0.4286. Acc.: 75.20%
Epoch: 974. Loss: 0.4211. Acc.: 76.25%
Epoch: 975. Loss: 0.4033. Acc.: 75.07%
Epoch: 976. Loss: 0.4102. Acc.: 74.93%
Epoch: 977. Loss: 0.4310. Acc.: 76.25%
Epoch: 978. Loss: 0.4232. Acc.: 75.07%
Epoch: 979. Loss: 0.4221. Acc.: 75.46%
Epoch: 980. Loss: 0.4421. Acc.: 74.93%
Epoch: 981. Loss: 0.4228. Acc.: 75.59%
Epoch: 982. Loss: 0.4239. Acc.: 75.46%
Epoch: 983. Loss: 0.4238. Acc.: 75.07%
Epoch: 984. Loss: 0.4327. Acc.: 76.38%
Epoch: 985. Loss: 0.4166. Acc.: 75.85%
Epoch: 986. Loss: 0.4239. Acc.: 77.56%
Epoch: 987. Loss: 0.4175. Acc.: 77.17%
Epoch: 988. Loss: 0.4185. Acc.: 75.33%
Epoch: 989. Loss: 0.4261. Acc.: 77.43%
Epoch: 990. Loss: 0.4286. Acc.: 75.98%
Epoch: 991. Loss: 0.4136. Acc.: 77.17%
Epoch: 992. Loss: 0.4212. Acc.: 75.72%
Epoch: 993. Loss: 0.4042. Acc.: 75.33%
Epoch: 994. Loss: 0.4173. Acc.: 74.41%
Epoch: 995. Los

Epoch: 1172. Loss: 0.4295. Acc.: 75.59%
Epoch: 1173. Loss: 0.4111. Acc.: 77.03%
Epoch: 1174. Loss: 0.4046. Acc.: 77.69%
Epoch: 1175. Loss: 0.4249. Acc.: 76.38%
Epoch: 1176. Loss: 0.4369. Acc.: 75.98%
Epoch: 1177. Loss: 0.4026. Acc.: 76.12%
Epoch: 1178. Loss: 0.4333. Acc.: 74.80%
Epoch: 1179. Loss: 0.4129. Acc.: 75.59%
Epoch: 1180. Loss: 0.4047. Acc.: 74.54%
Epoch: 1181. Loss: 0.4193. Acc.: 75.07%
Epoch: 1182. Loss: 0.4503. Acc.: 75.98%
Epoch: 1183. Loss: 0.4273. Acc.: 75.07%
Epoch: 1184. Loss: 0.4009. Acc.: 75.20%
Epoch: 1185. Loss: 0.4028. Acc.: 77.03%
Epoch: 1186. Loss: 0.4141. Acc.: 74.93%
Epoch: 1187. Loss: 0.4234. Acc.: 77.03%
Epoch: 1188. Loss: 0.4007. Acc.: 76.64%
Epoch: 1189. Loss: 0.4059. Acc.: 76.51%
Epoch: 1190. Loss: 0.4166. Acc.: 75.72%
Epoch: 1191. Loss: 0.4085. Acc.: 75.33%
Epoch: 1192. Loss: 0.4153. Acc.: 77.03%
Epoch: 1193. Loss: 0.4257. Acc.: 76.64%
Epoch: 1194. Loss: 0.4246. Acc.: 76.90%
Epoch: 1195. Loss: 0.4192. Acc.: 76.12%
Epoch: 1196. Loss: 0.4120. Acc.: 75.85%


Epoch: 1377. Loss: 0.4117. Acc.: 74.93%
Epoch: 1378. Loss: 0.3876. Acc.: 76.12%
Epoch: 1379. Loss: 0.3943. Acc.: 75.46%
Epoch: 1380. Loss: 0.3977. Acc.: 75.46%
Epoch: 1381. Loss: 0.3860. Acc.: 75.20%
Epoch: 1382. Loss: 0.4065. Acc.: 75.72%
Epoch: 1383. Loss: 0.3999. Acc.: 76.38%
Epoch: 1384. Loss: 0.3936. Acc.: 77.30%
Epoch: 1385. Loss: 0.3997. Acc.: 76.51%
Epoch: 1386. Loss: 0.3917. Acc.: 75.98%
Epoch: 1387. Loss: 0.4018. Acc.: 76.25%
Epoch: 1388. Loss: 0.4168. Acc.: 76.90%
Epoch: 1389. Loss: 0.4134. Acc.: 77.30%
Epoch: 1390. Loss: 0.4109. Acc.: 76.90%
Epoch: 1391. Loss: 0.4204. Acc.: 76.64%
Epoch: 1392. Loss: 0.3991. Acc.: 77.03%
Epoch: 1393. Loss: 0.3741. Acc.: 77.03%
Epoch: 1394. Loss: 0.3927. Acc.: 75.98%
Epoch: 1395. Loss: 0.4205. Acc.: 77.43%
Epoch: 1396. Loss: 0.4038. Acc.: 76.12%
Epoch: 1397. Loss: 0.4023. Acc.: 76.25%
Epoch: 1398. Loss: 0.4114. Acc.: 76.25%
Epoch: 1399. Loss: 0.3932. Acc.: 76.12%
Epoch: 1400. Loss: 0.4085. Acc.: 75.85%
Epoch: 1401. Loss: 0.4151. Acc.: 76.51%


Epoch: 1582. Loss: 0.3881. Acc.: 76.64%
Epoch: 1583. Loss: 0.3962. Acc.: 76.12%
Epoch: 1584. Loss: 0.4044. Acc.: 76.51%
Epoch: 1585. Loss: 0.4230. Acc.: 76.38%
Epoch: 1586. Loss: 0.4050. Acc.: 76.12%
Epoch: 1587. Loss: 0.3910. Acc.: 75.33%
Epoch: 1588. Loss: 0.4108. Acc.: 76.64%
Epoch: 1589. Loss: 0.4216. Acc.: 75.07%
Epoch: 1590. Loss: 0.3823. Acc.: 74.54%
Epoch: 1591. Loss: 0.3909. Acc.: 76.51%
Epoch: 1592. Loss: 0.4053. Acc.: 77.03%
Epoch: 1593. Loss: 0.3954. Acc.: 75.98%
Epoch: 1594. Loss: 0.3964. Acc.: 75.46%
Epoch: 1595. Loss: 0.3912. Acc.: 75.33%
Epoch: 1596. Loss: 0.3881. Acc.: 76.25%
Epoch: 1597. Loss: 0.4032. Acc.: 75.59%
Epoch: 1598. Loss: 0.4066. Acc.: 75.98%
Epoch: 1599. Loss: 0.3925. Acc.: 75.07%
Epoch: 1600. Loss: 0.3849. Acc.: 76.51%
Epoch: 1601. Loss: 0.3919. Acc.: 75.85%
Epoch: 1602. Loss: 0.3924. Acc.: 75.72%
Epoch: 1603. Loss: 0.3952. Acc.: 76.12%
Epoch: 1604. Loss: 0.3880. Acc.: 76.25%
Epoch: 1605. Loss: 0.4139. Acc.: 77.30%
Epoch: 1606. Loss: 0.3922. Acc.: 76.64%


<font size="5">**Compute result on testing data**</font><br>

In [ ]:
test_results = []
model.load_state_dict(torch.load('best.pth'))
model.eval()

# for batch in val_dl:
#     x_raw, y_batch = [t.to(device) for t in batch]
#     out = model(x_raw)
#     preds = F.log_softmax(out, dim=1).argmax(dim=1)
#     total += y_batch.size(0)
#     correct += (preds == y_batch).sum().item()
#     acc = correct / total
#     acc_history.append(acc)

# for batch in tst_dl:
#     x, y_batche = [t.to(device) for t in (batch)]
#     out = model(x)
#     y_hat = F.log_softmax(out, dim=1).argmax(dim=1)
#     test_results.extend(y_hat.tolist())
    
for batch in tst_dl:
        x_raw, y_batch = [t.to(device) for t in batch]
        out = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
acc = correct / total
print("accurancy on test data: "+str(acc))

<font size="5">**Store final result**</font><br>

In [ ]:
'''
submit = pd.read_csv(sample)
enc = joblib.load(ROOT/'encoder.model')
submit['surface'] = enc.inverse_transform(test_results)
submit.to_csv('submit_base.csv', index=None)
print("store result successfully!")
'''